In [1]:
import sys
# sys.path.insert(0, '../../Utilities/')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
#from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import time
# tf.enable_eager_execution()
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, X, u, layers, lb, ub):
        
        self.lb = lb
        self.ub = ub
        
        self.x = X[:,0:1]
        self.t = X[:,1:2]
        self.u = u
        
        self.layers = layers
        
        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        # Initialize parameters
        self.lambda_1 = tf.Variable([0.0], dtype=tf.float32)
        self.lambda_2 = tf.Variable([-6.0], dtype=tf.float32)
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
                
        self.u_pred = self.net_u(self.x_tf, self.t_tf)
        self.f_pred = self.net_f(self.x_tf, self.t_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        
#         self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
#                                                                 method = 'L-BFGS-B', 
#                                                                 options = {'maxiter': 50000,
#                                                                            'maxfun': 50000,
#                                                                            'maxcor': 50,
#                                                                            'maxls': 50,
#                                                                            'ftol' : 1.0 * np.finfo(float).eps})
    
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
            
    def net_u(self, x, t):  
        u = self.neural_net(tf.concat([x,t],1), self.weights, self.biases)
        return u
    
    def net_f(self, x, t):
        lambda_1 = self.lambda_1        
        lambda_2 = tf.exp(self.lambda_2)
        u = self.net_u(x,t)
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        f = u_t + lambda_1*u*u_x - lambda_2*u_xx
        
        return f
    
    def callback(self, loss, lambda_1, lambda_2):
        print('Loss: %e, l1: %.5f, l2: %.5f' % (loss, lambda_1, np.exp(lambda_2)))
        
        
    def train(self, nIter):
        tf_dict = {self.x_tf: self.x, self.t_tf: self.t, self.u_tf: self.u}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                lambda_1_value = self.sess.run(self.lambda_1)
                lambda_2_value = np.exp(self.sess.run(self.lambda_2))
                print('It: %d, Loss: %.3e, Lambda_1: %.3f, Lambda_2: %.6f, Time: %.2f' % 
                      (it, loss_value, lambda_1_value, lambda_2_value, elapsed))
                start_time = time.time()
        
#         self.optimizer_Adam.minimize(self.loss)
        
        
#         self.optimizer.minimize(self.sess,
#                                 feed_dict = tf_dict,
#                                 fetches = [self.loss, self.lambda_1, self.lambda_2],
#                                 loss_callback = self.callback)
        
        
    def predict(self, X_star):
        
        tf_dict = {self.x_tf: X_star[:,0:1], self.t_tf: X_star[:,1:2]}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        f_star = self.sess.run(self.f_pred, tf_dict)
        
        return u_star, f_star

In [3]:
if __name__ == "__main__": 
     
    nu = 0.01/np.pi

    N_u = 2000
    layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
    
    data = scipy.io.loadmat(r'C:\Users\Omkar\Downloads\burgers_shock.mat')
    
    t = data['t'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = np.real(data['usol']).T
    
    X, T = np.meshgrid(x,t)
    
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.flatten()[:,None]              

    # Doman bounds
    lb = X_star.min(0)
    ub = X_star.max(0)    
     
    ######################################################################
    ######################## Noiseles Data ###############################
    ######################################################################
    noise = 0.0            
             
    idx = np.random.choice(X_star.shape[0], N_u, replace=False)
    X_u_train = X_star[idx,:]
    u_train = u_star[idx,:]
    
    model = PhysicsInformedNN(X_u_train, u_train, layers, lb, ub)
    model.train(1000)
    
    u_pred, f_pred = model.predict(X_star)
            
    error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
    
    U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
        
    lambda_1_value = model.sess.run(model.lambda_1)
    lambda_2_value = model.sess.run(model.lambda_2)
    lambda_2_value = np.exp(lambda_2_value)
    
    error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
    error_lambda_2 = np.abs(lambda_2_value - nu)/nu * 100
    
    print('Error u: %e' % (error_u))    
    print('Error l1: %.5f%%' % (error_lambda_1))                             
    print('Error l2: %.5f%%' % (error_lambda_2))  

Device mapping:

It: 0, Loss: 3.437e-01, Lambda_1: 0.001, Lambda_2: 0.002479, Time: 0.89
It: 10, Loss: 2.439e-01, Lambda_1: -0.008, Lambda_2: 0.002491, Time: 0.17
It: 20, Loss: 2.252e-01, Lambda_1: -0.018, Lambda_2: 0.002517, Time: 0.15
It: 30, Loss: 2.045e-01, Lambda_1: -0.027, Lambda_2: 0.002543, Time: 0.14
It: 40, Loss: 1.636e-01, Lambda_1: -0.031, Lambda_2: 0.002550, Time: 0.15
It: 50, Loss: 1.218e-01, Lambda_1: -0.026, Lambda_2: 0.002518, Time: 0.14
It: 60, Loss: 8.982e-02, Lambda_1: -0.011, Lambda_2: 0.002477, Time: 0.15
It: 70, Loss: 7.008e-02, Lambda_1: 0.007, Lambda_2: 0.002432, Time: 0.13
It: 80, Loss: 5.735e-02, Lambda_1: 0.026, Lambda_2: 0.002385, Time: 0.13
It: 90, Loss: 4.937e-02, Lambda_1: 0.045, Lambda_2: 0.002342, Time: 0.13
It: 100, Loss: 4.547e-02, Lambda_1: 0.057, Lambda_2: 0.002311, Time: 0.13
It: 110, Loss: 4.286e-02, Lambda_1: 0.062, Lambda_2: 0.002294, Time: 0.13
It: 120, Loss: 4.163e-02, Lambda_1: 0.064, Lambda_2: 0.002287, Time: 0.13
It: 130, Loss: 4.075e-02, 

# Complete disection and analysis of PINN code on burgers equation

In [6]:
#loading and exploring data:
data = scipy.io.loadmat(r'C:\Users\Omkar\Downloads\burgers_shock.mat')

In [7]:
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

In [4]:
# time data : 100 data points equally spaced last element is 0.99
t

array([[0.  ],
       [0.01],
       [0.02],
       [0.03],
       [0.04],
       [0.05],
       [0.06],
       [0.07],
       [0.08],
       [0.09],
       [0.1 ],
       [0.11],
       [0.12],
       [0.13],
       [0.14],
       [0.15],
       [0.16],
       [0.17],
       [0.18],
       [0.19],
       [0.2 ],
       [0.21],
       [0.22],
       [0.23],
       [0.24],
       [0.25],
       [0.26],
       [0.27],
       [0.28],
       [0.29],
       [0.3 ],
       [0.31],
       [0.32],
       [0.33],
       [0.34],
       [0.35],
       [0.36],
       [0.37],
       [0.38],
       [0.39],
       [0.4 ],
       [0.41],
       [0.42],
       [0.43],
       [0.44],
       [0.45],
       [0.46],
       [0.47],
       [0.48],
       [0.49],
       [0.5 ],
       [0.51],
       [0.52],
       [0.53],
       [0.54],
       [0.55],
       [0.56],
       [0.57],
       [0.58],
       [0.59],
       [0.6 ],
       [0.61],
       [0.62],
       [0.63],
       [0.64],
       [0.65],
       [0.

In [7]:
#shape of time data
t.shape

(100, 1)

In [8]:
# position data : 256 data points equally spaced between -1 to 1
x

array([[-1.        ],
       [-0.99215686],
       [-0.98431373],
       [-0.97647059],
       [-0.96862745],
       [-0.96078431],
       [-0.95294118],
       [-0.94509804],
       [-0.9372549 ],
       [-0.92941176],
       [-0.92156863],
       [-0.91372549],
       [-0.90588235],
       [-0.89803922],
       [-0.89019608],
       [-0.88235294],
       [-0.8745098 ],
       [-0.86666667],
       [-0.85882353],
       [-0.85098039],
       [-0.84313725],
       [-0.83529412],
       [-0.82745098],
       [-0.81960784],
       [-0.81176471],
       [-0.80392157],
       [-0.79607843],
       [-0.78823529],
       [-0.78039216],
       [-0.77254902],
       [-0.76470588],
       [-0.75686275],
       [-0.74901961],
       [-0.74117647],
       [-0.73333333],
       [-0.7254902 ],
       [-0.71764706],
       [-0.70980392],
       [-0.70196078],
       [-0.69411765],
       [-0.68627451],
       [-0.67843137],
       [-0.67058824],
       [-0.6627451 ],
       [-0.65490196],
       [-0

In [9]:
#shape of x
x.shape

(256, 1)

In [19]:
#solution data
Exact

array([[ 1.22464680e-16,  2.46374492e-02,  4.92599411e-02, ...,
        -4.92599411e-02, -2.46374492e-02, -1.22464680e-16],
       [ 2.95362215e-17,  2.38801772e-02,  4.77471404e-02, ...,
        -4.77471404e-02, -2.38801772e-02, -1.08544418e-16],
       [ 1.08470836e-16,  2.31684474e-02,  4.63251758e-02, ...,
        -4.63251758e-02, -2.31684474e-02, -1.29376253e-16],
       ...,
       [ 3.57297976e-16,  6.07646192e-03,  1.21528662e-02, ...,
        -1.21528662e-02, -6.07646192e-03,  1.04877526e-16],
       [ 2.61833228e-16,  6.02971754e-03,  1.20593792e-02, ...,
        -1.20593792e-02, -6.02971754e-03,  2.17508692e-16],
       [ 9.39536897e-17,  5.98368729e-03,  1.19673204e-02, ...,
        -1.19673204e-02, -5.98368729e-03,  1.12388795e-16]])

In [11]:
#Shape of solution data u(t,x)
Exact.shape

(100, 256)

In [8]:
#discovering the function
X, T = np.meshgrid(x,t)

In [30]:
# X has a shape of (100,256) and all rows are the same that is  256 x- values -1 to 1 written 100 times 
X

array([[-1.        , -0.99215686, -0.98431373, ...,  0.98431373,
         0.99215686,  1.        ],
       [-1.        , -0.99215686, -0.98431373, ...,  0.98431373,
         0.99215686,  1.        ],
       [-1.        , -0.99215686, -0.98431373, ...,  0.98431373,
         0.99215686,  1.        ],
       ...,
       [-1.        , -0.99215686, -0.98431373, ...,  0.98431373,
         0.99215686,  1.        ],
       [-1.        , -0.99215686, -0.98431373, ...,  0.98431373,
         0.99215686,  1.        ],
       [-1.        , -0.99215686, -0.98431373, ...,  0.98431373,
         0.99215686,  1.        ]])

In [34]:
# this has shape (100,256) aswell but as we keep traversing the rows the value increments. So T[0] is all 0s 256 times,
#T[1] is 0.01 written 256 times and so on
T

array([[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       ...,
       [0.97, 0.97, 0.97, ..., 0.97, 0.97, 0.97],
       [0.98, 0.98, 0.98, ..., 0.98, 0.98, 0.98],
       [0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99]])

In [9]:
#making of X_star an u_star
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]

In [42]:
#X_star has shape (25600,2) so (256*100,2), the first row is [-1,0] second is [-0.99,0] and this goes on till u have
# [1,0] - the 255th index then at 256 index the x resets to -1 and t becomes 0.01 and this is the matrix so on 
X_star

array([[-1.        ,  0.        ],
       [-0.99215686,  0.        ],
       [-0.98431373,  0.        ],
       ...,
       [ 0.98431373,  0.99      ],
       [ 0.99215686,  0.99      ],
       [ 1.        ,  0.99      ]])

In [46]:
#shape of (25600,1) this gives the corresponding solution (u) values for the above X_star. So 
#the 0th index is the solution at [-1,0] written in (x,t) order and the solution is u[0] Note all of these till now 
# are nparrays
u_star

array([[ 1.22464680e-16],
       [ 2.46374492e-02],
       [ 4.92599411e-02],
       ...,
       [-1.19673204e-02],
       [-5.98368729e-03],
       [ 1.12388795e-16]])

In [10]:
# Domain bounds
lb = X_star.min(0).astype('float32') # [-1,0]
ub = X_star.max(0).astype('float32') # [1,0.99]

In [11]:
#making of training data 
N_u = 2000 # no. of training samples from data
idx = np.random.choice(X_star.shape[0], N_u, replace=False) # gives random index array
X_u_train = X_star[idx,:] # shape (N_u,2) in the format (x,t)
u_train = u_star[idx,:] # shape (2000,1) the solution to those 2000 values of x,t

In [17]:
X_u_train=X_u_train.astype('float32')

## Understanding whats inside the PINN

In [7]:
x = X_u_train[:,0:1] #- takes the X column from train data and outputs (2000,1) shape
t = X_u_train[:,1:2] #- takes the t column from train data and outputs (2000,1) shape
u = u_train # - takes the u from train data and outputs (2000,1) shape
# self.layers = layers # gets the layers list (NOT AN NP ARRAY)

In [54]:
#creating the Session and its variables n losses
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
# Initialize parameters
lambda_1 = tf.Variable([0.0], dtype=tf.float32)
lambda_2 = tf.Variable([-6.0], dtype=tf.float32)

x_tf = tf.placeholder(tf.float32, shape=[None,1])
t_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_tf = tf.placeholder(tf.float32, shape=[None, 1])

u_pred = net_u(x_tf, t_tf)
f_pred = net_f(x_tf, t_tf)

loss = tf.reduce_mean(tf.square(u_tf - u_pred)) + \
            tf.reduce_mean(tf.square(f_pred))

optimizer = tf.contrib.opt.ScipyOptimizerInterface(loss, 
                                                        method = 'L-BFGS-B', 
                                                        options = {'maxiter': 50000,
                                                                   'maxfun': 50000,
                                                                   'maxcor': 50,
                                                                   'maxls': 50,
                                                                   'ftol' : 1.0 * np.finfo(float).eps})

optimizer_Adam = tf.train.AdamOptimizer()
train_op_Adam = optimizer_Adam.minimize(loss)

init = tf.global_variables_initializer()
sess.run(init)

Device mapping:

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [27]:
#size is a list 
def xavier_init(size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32) 
                #returns a tf.Variable

In [70]:
#example
a=xavier_init([1,2])

In [20]:
#running the code
init = tf.global_variables_initializer() # prepare an init node
with tf.Session() as sess:
    init.run() # actually initialize all the variables
    result = a.eval()
    print(result)
    #this will show the actual numbers output

NameError: name 'a' is not defined

In [28]:
#this returns the neural network weights and biases. NOTE both weights and biases are lists
def initialize_NN(layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases

In [11]:
#example code to run initializeNN
# layers = [1,2,1]
layers=[2,20,20,20,20,20,20,20,20,1]
w , b= initialize_NN(layers)

In [25]:
#running the code
print(w) # this is a list of tf.Variables
print(b) # this is a list of tf.Variables

[<tf.Variable 'Variable_14:0' shape=(1, 2) dtype=float32_ref>, <tf.Variable 'Variable_16:0' shape=(2, 1) dtype=float32_ref>]
[<tf.Variable 'Variable_15:0' shape=(1, 2) dtype=float32_ref>, <tf.Variable 'Variable_17:0' shape=(1, 1) dtype=float32_ref>]


In [14]:
#running the code
init = tf.global_variables_initializer() # prepare an init node
with tf.Session() as sess:
    init.run() # actually initialize all the variables
    W1 = w[0].eval() # this will output the weights of the first element in the weights list 
    B1= b[0].eval() # this will output the bias of the first element in the bias list 
    print(W1)
    print(B1)
    #this will show the actual numbers output

[[ 0.03307999  0.00552576 -0.14778398  0.1631586  -0.08150904 -0.15934165
   0.07438523  0.45788297 -0.22241917  0.02204545 -0.28173542 -0.26593822
  -0.17192322  0.23722447  0.54474473 -0.5335447  -0.04868652 -0.11398798
  -0.06433471  0.06431513]
 [-0.03353772  0.12033412 -0.09941009  0.00816181 -0.30426386  0.02664434
   0.0694659  -0.33017966 -0.08288395 -0.21282418  0.21034858  0.13623093
  -0.22780806 -0.47640064 -0.11849657 -0.37377158 -0.24949227  0.13145612
  -0.16492261 -0.10654896]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [46]:
def neural_net(X, weights, biases):
        num_layers = len(weights) + 1
        
        H = 2.0*(X - lb)/(ub - lb) - 1.0  # this normalizes the input values of X
        H=tf.cast(H , dtype=tf.float32)  #type casts cuz x might be float64
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
#             H = tf.math.sigmoid(tf.add(tf.matmul(H, W), b))
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y # this is the final output vector [N_u,1]that the neural network outputs after
                 #complete forward propogation

In [70]:
#running the code
init = tf.global_variables_initializer() # prepare an init node
with tf.Session() as sess:
    init.run() # actually initialize all the variables
    result = arr.eval()
    print(result)
    #this will show the actual numbers output

[[1.]
 [2.]
 [3.]]


In [30]:
#making vanilla network
def net_u(x,t):  
        u = neural_net(tf.concat([x,t],1),w,b) # these weights and Biases are the dummy ones i made on top
        return u # this basically returns the forward prop values. All we are doing is creating a NN called u

In [55]:
def net_f(x, t):
        lambda_1_ = lambda_1        
        lambda_2_ = tf.exp(lambda_2)
        u = net_u(x,t)
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        f = u_t + lambda_1_*u*u_x - tf.exp(lambda_2_)*u_xx
        
        return f

In [59]:
def callback(loss, lambda_1, lambda_2):
        print('Loss: %e, l1: %.5f, l2: %.5f' % (loss, lambda_1, np.exp(lambda_2)))

In [62]:
def train(nIter):
        tf_dict = {x_tf: X_u_train[:,0:1], t_tf: X_u_train[:,1:2], u_tf: u_train}
        
        start_time = time.time()
        for it in range(nIter):
            sess.run(train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = sess.run(loss, tf_dict)
                lambda_1_value = sess.run(lambda_1)
                lambda_2_value = np.exp(sess.run(lambda_2))
                print('It: %d, Loss: %.3e, Lambda_1: %.3f, Lambda_2: %.6f, Time: %.2f' % 
                      (it, loss_value, lambda_1_value, lambda_2_value, elapsed))
                start_time = time.time()
        
        optimizer.minimize(sess,
                                feed_dict = tf_dict,
                                fetches = [loss, lambda_1, lambda_2],
                                loss_callback = callback)

# Building and testing PINN for Rod Data


In [14]:
#making of Rod data Set
dataframe1 = pd.read_csv(r"C:\Users\Omkar\Downloads\Rod data.csv")
x_values=dataframe1.loc[:,"Length [mm]"] 
x_values=np.array(x_values)
x_values=x_values.reshape(257,1)
T_values=dataframe1.loc[:,"Value [K]"] 
T_values=np.array(T_values)
T_values=T_values.reshape(257,1)
x_star=x_values.astype('float32')
x_tensor=tf.Variable(x_star, dtype = tf.float32)
T_star=T_values.astype('float32')
T_tensor=tf.Variable(T_star, dtype = tf.float32)
lb = np.array([0.]).astype('float32')
ub = np.array([0.3]).astype('float32')

In [15]:
T_star

array([[1800.  ],
       [1794.1 ],
       [1788.3 ],
       [1782.4 ],
       [1776.6 ],
       [1770.7 ],
       [1764.8 ],
       [1759.  ],
       [1753.1 ],
       [1747.3 ],
       [1741.4 ],
       [1735.5 ],
       [1729.7 ],
       [1723.8 ],
       [1718.  ],
       [1712.1 ],
       [1706.2 ],
       [1700.4 ],
       [1694.5 ],
       [1688.7 ],
       [1682.8 ],
       [1677.  ],
       [1671.1 ],
       [1665.2 ],
       [1659.4 ],
       [1653.5 ],
       [1647.7 ],
       [1641.8 ],
       [1635.9 ],
       [1630.1 ],
       [1624.2 ],
       [1618.4 ],
       [1612.5 ],
       [1606.6 ],
       [1600.8 ],
       [1594.9 ],
       [1589.1 ],
       [1583.2 ],
       [1577.3 ],
       [1571.5 ],
       [1565.6 ],
       [1559.8 ],
       [1553.9 ],
       [1548.  ],
       [1542.2 ],
       [1536.3 ],
       [1530.5 ],
       [1524.6 ],
       [1518.7 ],
       [1512.9 ],
       [1507.  ],
       [1501.2 ],
       [1495.3 ],
       [1489.5 ],
       [1483.6 ],
       [14

In [ ]:
#creating the Session and its variables n losses
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
# Initialize parameters
lambda_1 = tf.Variable([0.0], dtype=tf.float32) #Making lambda 1 value 

x_tf = tf.placeholder(tf.float32, shape=[None,1])
u_tf = tf.placeholder(tf.float32, shape=[None, 1])

u_pred = net_u(x_tf)
f_pred = net_f(x_tf)

loss = tf.reduce_mean(tf.square(u_tf - u_pred)) + tf.reduce_mean(tf.square(f_pred))

optimizer = tf.contrib.opt.ScipyOptimizerInterface(loss, 
                                                        method = 'L-BFGS-B', 
                                                        options = {'maxiter': 50000,
                                                                   'maxfun': 50000,
                                                                   'maxcor': 50,
                                                                   'maxls': 50,
                                                                   'ftol' : 1.0 * np.finfo(float).eps})

optimizer_Adam = tf.train.AdamOptimizer()
train_op_Adam = optimizer_Adam.minimize(loss)

init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
#size is a list 
def xavier_init(size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32) 
                #returns a tf.Variable

In [ ]:
#this returns the neural network weights and biases. NOTE both weights and biases are lists
def initialize_NN(layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases

In [20]:
#This is an integral step and will be explained in detail
def neural_net(X, weights, biases):
        num_layers = len(weights) + 1
        
#         H = 2.0*(X - lb)/(ub - lb) - 1.0
        H = (X - lb)/(ub - lb) # this normalizes the input values of X
#         H=H.astype('float32')  #type casts cuz x might be float64
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.math.sigmoid(tf.add(tf.matmul(H, W), b))
#             H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y # this is the final output vector [N_u,1]that the neural network outputs after
                 #complete forward propogation

In [ ]:
#making vanilla network
def net_u(x):  
        u = neural_net(tf.concat([x],1),w,b) # these weights and Biases are the dummy ones i made on top
        return u # this basically returns the forward prop values. All we are doing is creating a NN called u

In [8]:
#making PINN
#set lambda_1
# lambda_1=tf.Variable([0.0], dtype=tf.float32)
def net_f(x):
#         lambda_1 = self.lambda_1        
#         lambda_2 = tf.exp(self.lambda_2)
        u = net_u(x)
#         u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        f = -1*lambda_1*u_xx
        
        return f

In [13]:
#This is only for one dependent variable x
#lets take random X of shape (3,1) for simplicity with data 1 2 3 column vector
# X=np.array([1.0,2.0,3.0]).reshape(3,1)
# arr = X.astype('float32')
arr=tf.Variable([[1.0],[2.0],[3.0]],dtype=tf.float32,shape=(3,1)) #making it a tensor
#we will take an assumed set of weights and biases as well for a small layers size of [1,2,1]
var1 =tf.Variable([[1.11,-0.98]], shape=(1,2), dtype=tf.float32)
var2=tf.Variable([[0.31],[ 0.68]] , shape=(2,1),dtype=tf.float32)
b1=tf.Variable([[0.0,0.0]], shape=(1,2),dtype=tf.float32)
b2=tf.Variable([[0.0]], shape=(1,1), dtype=tf.float32)
W=[var1 , var2]
B=[b1 , b2]
lb=np.array([1.0],dtype='float32')
ub=np.array([3.0],dtype='float32')

In [10]:
def callback( loss, lambda_1):
        print('Loss: %e, l1: %.5f' % (loss, lambda_1))

In [23]:
#understanding training of network:
def train(nIter):
        tf_dict = {x_tf: x_star, u_tf: scaled_T}
        
        start_time = time.time()
        for it in range(nIter):
            sess.run(train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = sess.run(loss, tf_dict)
                lambda_1_value = sess.run(lambda_1)
#                 lambda_2_value = np.exp(self.sess.run(self.lambda_2))
                print('It: %d, Loss: %.3e, Lambda_1: %.3f, Time: %.2f' % 
                      (it, loss_value, lambda_1_value, elapsed))
                start_time = time.time()
        
        optimizer.minimize(loss)

In [36]:
train(3000)

It: 0, Loss: 2.231e-01, Lambda_1: 10.000, Time: 0.76
It: 10, Loss: 9.612e-02, Lambda_1: 10.000, Time: 0.05
It: 20, Loss: 8.704e-02, Lambda_1: 9.998, Time: 0.03
It: 30, Loss: 8.835e-02, Lambda_1: 9.996, Time: 0.03
It: 40, Loss: 8.401e-02, Lambda_1: 9.993, Time: 0.03
It: 50, Loss: 8.450e-02, Lambda_1: 9.990, Time: 0.03
It: 60, Loss: 8.403e-02, Lambda_1: 9.986, Time: 0.03
It: 70, Loss: 8.403e-02, Lambda_1: 9.981, Time: 0.04
It: 80, Loss: 8.399e-02, Lambda_1: 9.975, Time: 0.04
It: 90, Loss: 8.398e-02, Lambda_1: 9.967, Time: 0.03
It: 100, Loss: 8.397e-02, Lambda_1: 9.958, Time: 0.03
It: 110, Loss: 8.397e-02, Lambda_1: 9.948, Time: 0.03
It: 120, Loss: 8.397e-02, Lambda_1: 9.936, Time: 0.03
It: 130, Loss: 8.397e-02, Lambda_1: 9.924, Time: 0.03
It: 140, Loss: 8.397e-02, Lambda_1: 9.910, Time: 0.03
It: 150, Loss: 8.397e-02, Lambda_1: 9.895, Time: 0.04
It: 160, Loss: 8.396e-02, Lambda_1: 9.879, Time: 0.04
It: 170, Loss: 8.396e-02, Lambda_1: 9.862, Time: 0.03
It: 180, Loss: 8.396e-02, Lambda_1: 9

It: 1510, Loss: 2.740e-02, Lambda_1: 6.893, Time: 0.04
It: 1520, Loss: 3.173e-02, Lambda_1: 6.873, Time: 0.04
It: 1530, Loss: 3.010e-02, Lambda_1: 6.850, Time: 0.04
It: 1540, Loss: 2.448e-02, Lambda_1: 6.834, Time: 0.03
It: 1550, Loss: 2.462e-02, Lambda_1: 6.821, Time: 0.03
It: 1560, Loss: 2.393e-02, Lambda_1: 6.812, Time: 0.04
It: 1570, Loss: 2.333e-02, Lambda_1: 6.803, Time: 0.04
It: 1580, Loss: 2.279e-02, Lambda_1: 6.794, Time: 0.03
It: 1590, Loss: 2.236e-02, Lambda_1: 6.786, Time: 0.04
It: 1600, Loss: 2.199e-02, Lambda_1: 6.778, Time: 0.03
It: 1610, Loss: 2.799e-02, Lambda_1: 6.769, Time: 0.03
It: 1620, Loss: 3.532e-02, Lambda_1: 6.747, Time: 0.03
It: 1630, Loss: 2.158e-02, Lambda_1: 6.724, Time: 0.03
It: 1640, Loss: 2.209e-02, Lambda_1: 6.708, Time: 0.03
It: 1650, Loss: 2.074e-02, Lambda_1: 6.697, Time: 0.03
It: 1660, Loss: 2.037e-02, Lambda_1: 6.689, Time: 0.03
It: 1670, Loss: 1.981e-02, Lambda_1: 6.681, Time: 0.03
It: 1680, Loss: 1.941e-02, Lambda_1: 6.674, Time: 0.03
It: 1690, 

In [185]:
mini=tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [186]:
i=tf.global_variables_initializer()
s=tf.Session()
s.run(i)

In [6]:
scaler = MinMaxScaler()
model1 = scaler.fit(x_star)
scaled_x = model1.transform(x_star)

In [7]:
model2 = scaler.fit(T_star)
scaled_T = model2.transform(T_star)

In [25]:
scaled_T

array([[1.        ],
       [0.99606667],
       [0.9922    ],
       [0.98826667],
       [0.9844    ],
       [0.98046667],
       [0.97653333],
       [0.97266667],
       [0.96873333],
       [0.96486667],
       [0.96093333],
       [0.957     ],
       [0.95313333],
       [0.9492    ],
       [0.94533333],
       [0.9414    ],
       [0.93746667],
       [0.9336    ],
       [0.92966667],
       [0.9258    ],
       [0.92186667],
       [0.918     ],
       [0.91406667],
       [0.91013333],
       [0.90626667],
       [0.90233333],
       [0.89846667],
       [0.89453333],
       [0.8906    ],
       [0.88673333],
       [0.8828    ],
       [0.87893333],
       [0.875     ],
       [0.87106667],
       [0.8672    ],
       [0.86326667],
       [0.8594    ],
       [0.85546667],
       [0.85153333],
       [0.84766667],
       [0.84373333],
       [0.83986667],
       [0.83593333],
       [0.832     ],
       [0.82813333],
       [0.8242    ],
       [0.82033333],
       [0.816

In [187]:
fd = {x_tf: scaled_x , u_tf: scaled_T}
for i in range(1000):
    print("lol u got error" , s.run(loss,fd))
    s.run(mini,fd)
    print("current" , s.run(loss,fd))

lol u got error 0.20335515
current 0.20018157
lol u got error 0.20018157
current 0.1970922
lol u got error 0.1970922
current 0.1940849
lol u got error 0.1940849
current 0.19115739
lol u got error 0.19115739
current 0.18830764
lol u got error 0.18830764
current 0.18553357
lol u got error 0.18553357
current 0.18283315
lol u got error 0.18283315
current 0.18020442
lol u got error 0.18020442
current 0.17764552
lol u got error 0.17764552
current 0.17515454
lol u got error 0.17515454
current 0.17272975
lol u got error 0.17272975
current 0.17036937
lol u got error 0.17036937
current 0.16807163
lol u got error 0.16807163
current 0.165835
lol u got error 0.165835
current 0.1636577
lol u got error 0.1636577
current 0.16153829
lol u got error 0.16153829
current 0.15947516
lol u got error 0.15947516
current 0.15746686
lol u got error 0.15746686
current 0.15551187
lol u got error 0.15551187
current 0.15360887
lol u got error 0.15360887
current 0.1517564
lol u got error 0.1517564
current 0.14995317


current 0.084907725
lol u got error 0.084907725
current 0.084883094
lol u got error 0.084883094
current 0.08485913
lol u got error 0.08485913
current 0.084835805
lol u got error 0.084835805
current 0.08481309
lol u got error 0.08481309
current 0.084790975
lol u got error 0.084790975
current 0.084769465
lol u got error 0.084769465
current 0.08474854
lol u got error 0.08474854
current 0.08472816
lol u got error 0.08472816
current 0.0847083
lol u got error 0.0847083
current 0.084689006
lol u got error 0.084689006
current 0.084670216
lol u got error 0.084670216
current 0.084651925
lol u got error 0.084651925
current 0.08463412
lol u got error 0.08463412
current 0.08461678
lol u got error 0.08461678
current 0.08459993
lol u got error 0.08459993
current 0.08458352
lol u got error 0.08458352
current 0.08456753
lol u got error 0.08456753
current 0.08455198
lol u got error 0.08455198
current 0.08453685
lol u got error 0.08453685
current 0.08452212
lol u got error 0.08452212
current 0.08450778
l

lol u got error 0.083991185
current 0.08399102
lol u got error 0.08399102
current 0.08399085
lol u got error 0.08399085
current 0.08399069
lol u got error 0.08399069
current 0.08399055
lol u got error 0.08399055
current 0.08399039
lol u got error 0.08399039
current 0.08399026
lol u got error 0.08399026
current 0.08399011
lol u got error 0.08399011
current 0.08398998
lol u got error 0.08398998
current 0.08398984
lol u got error 0.08398984
current 0.08398972
lol u got error 0.08398972
current 0.083989576
lol u got error 0.083989576
current 0.083989464
lol u got error 0.083989464
current 0.08398934
lol u got error 0.08398934
current 0.083989225
lol u got error 0.083989225
current 0.08398913
lol u got error 0.08398913
current 0.08398902
lol u got error 0.08398902
current 0.08398889
lol u got error 0.08398889
current 0.08398879
lol u got error 0.08398879
current 0.0839887
lol u got error 0.0839887
current 0.08398861
lol u got error 0.08398861
current 0.0839885
lol u got error 0.0839885
curr

lol u got error 0.08398505
current 0.08398505
lol u got error 0.08398505
current 0.083985046
lol u got error 0.083985046
current 0.083985046
lol u got error 0.083985046
current 0.083985046
lol u got error 0.083985046
current 0.083985046
lol u got error 0.083985046
current 0.083985046
lol u got error 0.083985046
current 0.083985046
lol u got error 0.083985046
current 0.083985046
lol u got error 0.083985046
current 0.08398504
lol u got error 0.08398504
current 0.08398503
lol u got error 0.08398503
current 0.08398504
lol u got error 0.08398504
current 0.08398504
lol u got error 0.08398504
current 0.08398503
lol u got error 0.08398503
current 0.08398503
lol u got error 0.08398503
current 0.08398503
lol u got error 0.08398503
current 0.08398503
lol u got error 0.08398503
current 0.08398503
lol u got error 0.08398503
current 0.08398503
lol u got error 0.08398503
current 0.08398503
lol u got error 0.08398503
current 0.08398504
lol u got error 0.08398504
current 0.08398503
lol u got error 0.08

lol u got error 0.08398499
current 0.08398499
lol u got error 0.08398499
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.08398499
lol u got error 0.08398499
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.08398501
lol u got error 0.08398501
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.08398501
lol u got error 0.08398501
current 0.08398501
lol u got error 0.08398501
current 0.08398499
lol u got error 0.08398499
current 0.08398499
lol u got error 0.08398499
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.083985
lol u got error 0.083985
current 0.083985
lol 

current 0.083985
lol u got error 0.083985
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.08398499
lol u got error 0.08398499
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.083985
lol u got error 0.083985
current 0.083984986
lol u got error 0.083984986
current 0.08398499
lol u got error 0.08398499
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.083985
lol u got error 0.083985
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.083984986
lol u got error 0.083984986
current 0.083985
lol u got error 0.083985
current 0.083985
lol

In [188]:
s.run(lambda_1)

array([5.], dtype=float32)

In [35]:
lolz=[]

In [36]:

for i in range(0,257):
    lolz.append(8.0)
    

In [38]:
k_= np.array(lolz).reshape(257,1)

In [43]:
k=tf.Variable(k_ , dtype=tf.float32)

In [44]:
k

<tf.Variable 'Variable_256:0' shape=(257, 1) dtype=float32_ref>

In [54]:
def suh(x):
    print("suh")

suh


In [ ]:
#for 1d Rod simulation

In [46]:
class PhysicsInformedNN2:
    # Initialize the class
    def __init__(self, X, u, layers, lb, ub):
        
        self.lb = lb
        self.ub = ub
        
        self.x = X[:,0:1]
#         self.t = X[:,1:2]
        self.u = u
        
        self.layers = layers
        
        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))

        # Initialize parameters
        self.lambda_1 = tf.Variable([1.0], dtype=tf.float32)
#         self.lambda_2 = tf.Variable([-6.0], dtype=tf.float32)
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
#         self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
                
        self.u_pred = self.net_u(self.x_tf)
        self.f_pred = self.net_f(self.x_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        
#         self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
#                                                                 method = 'L-BFGS-B', 
#                                                                 options = {'maxiter': 50000,
#                                                                            'maxfun': 50000,
#                                                                            'maxcor': 50,
#                                                                            'maxls': 50,
#                                                                            'ftol' : 1.0 * np.finfo(float).eps})
    
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        H = (X - self.lb)/(self.ub - self.lb)
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.math.sigmoid(tf.add(tf.matmul(H, W), b))
#             H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
    
        
            
    def net_u(self, x):  
        u = self.neural_net(tf.concat([x],1), self.weights, self.biases)
        return u
    
    def net_f(self, x):
        lambda_1 = self.lambda_1       
#         lambda_2 = tf.exp(self.lambda_2)
        u = self.net_u(x)
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        f= (lambda_1+1)*u_xx
#         f = -1*lambda_1*u_xx 
#         f = -1*lambda_1*u_xx
        
        return f
    
    def callback(self, loss, lambda_1):
        print('Loss: %e, l1: %.5f' % (loss, lambda_1))
        
        
    def train(self, nIter):
        tf_dict = {self.x_tf: self.x, self.u_tf: self.u}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                lambda_1_value = self.sess.run(self.lambda_1)
#                 lambda_2_value = np.exp(self.sess.run(self.lambda_2))
                print('It: %d, Loss: %.3e, Lambda_1: %.3f, Time: %.2f' % 
                      (it, loss_value, lambda_1_value, elapsed))
                u_pred_x = tf.gradients(self.sess.run(self.u_pred,tf_dict),self.sess.run(self.x_tf,tf_dict))[0]
#                 u_pred_xx = tf.gradients(u_pred_x,self.sess.run(self.x_tf,tf_dict))[0]
                
                print(u_pred_x.eval())
                print((self.net_f(self.sess.run(self.x_tf,tf_dict))))
                
#                 print(self.sess.run(self.u_tf,tf_dict) ," " , self.sess.run(self.u_pred,tf_dict))
                start_time = time.time()
        
#         self.optimizer.minimize(self.sess,
#                                 feed_dict = tf_dict,
#                                 fetches = [self.loss, self.lambda_1],
#                                 loss_callback = self.callback)
        
        
    def predict(self, X_star):
        
        tf_dict = {self.x_tf: X_star}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        f_star = self.sess.run(self.f_pred, tf_dict)
        
        return u_star, f_star

In [4]:
layers2 = [1, 20, 20, 20, 20, 20, 20, 20, 20, 1]

In [47]:
model = PhysicsInformedNN2(scaled_x,scaled_T, layers2, lb, ub)
model.train(100)

Device mapping:

It: 0, Loss: 5.064e-01, Lambda_1: 1.000, Time: 5.11


AttributeError: 'NoneType' object has no attribute 'eval'

In [58]:
a,b=model.predict(scaled_x)

In [60]:
b

array([[-3.32133472e-03],
       [-3.06120515e-03],
       [-2.81503797e-03],
       [-2.58164108e-03],
       [-2.36158073e-03],
       [-2.15317309e-03],
       [-1.95658207e-03],
       [-1.77113712e-03],
       [-1.59572065e-03],
       [-1.43049657e-03],
       [-1.27533078e-03],
       [-1.12909079e-03],
       [-9.91433859e-04],
       [-8.61838460e-04],
       [-7.40408897e-04],
       [-6.26236200e-04],
       [-5.19022346e-04],
       [-4.18886542e-04],
       [-3.25053930e-04],
       [-2.36958265e-04],
       [-1.54539943e-04],
       [-7.75754452e-05],
       [-5.60283661e-06],
       [ 6.14523888e-05],
       [ 1.24335289e-04],
       [ 1.82881951e-04],
       [ 2.37077475e-04],
       [ 2.87979841e-04],
       [ 3.35037708e-04],
       [ 3.78921628e-04],
       [ 4.20004129e-04],
       [ 4.57659364e-04],
       [ 4.92990017e-04],
       [ 5.25474548e-04],
       [ 5.55798411e-04],
       [ 5.83872199e-04],
       [ 6.09889627e-04],
       [ 6.34267926e-04],
       [ 6.5

In [76]:
import pandas as pd

In [77]:
dataframe1 = pd.read_csv(r"C:\Users\Omkar\Downloads\Rod data.csv")
 
# print(dataframe1)

In [78]:
x_values=dataframe1.loc[:,"Length [mm]"]  

In [79]:
x_values=np.array(x_values)

In [80]:
x_values=x_values.reshape(257,1)

In [81]:
x_values=x_values.astype('float32')

In [82]:
T_values=dataframe1.loc[:,"Value [K]"] 

In [83]:
T_values=np.array(T_values)

In [84]:
T_values=T_values.reshape(257,1)

In [86]:
x_star=x_values

In [87]:
x_star=tf.Variable(x_star, dtype = tf.float32)

In [90]:
T_star=T_values

In [91]:
T_star=tf.Variable(T_star, dtype = tf.float32)

In [100]:
z=tf.Variable([0])

In [102]:
#running the code
init = tf.global_variables_initializer() # prepare an init node
with tf.Session() as sess:
    init.run() # actually initialize all the variables
    result = z.eval()
    print(result)
    #this will show the actual numbers output

[0]


In [101]:
a=z.numpy()

AttributeError: 'RefVariable' object has no attribute 'numpy'

In [ ]:
#normalizing data

In [15]:
scaler = MinMaxScaler()
model1 = scaler.fit(x_star)
scaled_x = model1.transform(x_star)

In [16]:
model2 = scaler.fit(T_star)
scaled_T = model2.transform(T_star)

In [19]:
scaled_x.shape

(257, 1)

In [20]:
scaled_T.shape

(257, 1)

In [92]:
N=200
layers = [1, 20, 20, 20, 20, 20, 20, 20, 20, 1]
# layers = [1 , 2 ,1]

In [94]:
lb = tf.Variable([0.0],dtype=tf.float32)
ub = tf.Variable([0.3],dtype=tf.float32) 

In [135]:
x_star.max(0) 

array([0.3])

In [134]:
x_star.min(0)

array([0.])

In [24]:
idx_ = np.random.choice(scaled_x.shape[0], N, replace=False)
x_u_train = scaled_x[idx_,:]
T_u_train = scaled_T[idx_,:]

In [25]:
x_u_train[:,0:1].shape

(200, 1)

In [26]:
T_u_train.shape

(200, 1)

In [ ]:
#PINN for rod

In [84]:
#understading the code

In [ ]:
#xaviers init

In [21]:
#sample data x_star T_star

In [2]:
def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]        
    xavier_stddev = np.sqrt(2/(in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)

In [3]:
def initialize_NN(layers):
    weights = []
    biases = []
    num_layers = len(layers) 
    for l in range(0,num_layers-1):
        W = xavier_init(size=[layers[l], layers[l+1]])
        b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
        weights.append(W)
        biases.append(b)        
    return weights, biases

In [81]:
xavier_init([2,1])

<tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[-0.2455011],
       [ 0.8066156]], dtype=float32)>

In [100]:
tf.truncated_normal([1, 2], stddev=1)

<tf.Tensor 'truncated_normal_104:0' shape=(1, 2) dtype=float32>

In [99]:
tf.truncated_normal([2])

<tf.Tensor 'truncated_normal_103:0' shape=(2,) dtype=float32>

In [146]:
x,y=initialize_NN([1,2,1])

In [90]:
x

[<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[-0.62955666,  1.1757891 ]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
 array([[ 0.8035322 ],
        [-0.47300494]], dtype=float32)>]

In [ ]:
#neural net

In [140]:
def neural_net( X, weights, biases):
    num_layers = len(weights) + 1

    H = 2.0*(X - lb)/(ub - lb) - 1.0
    for l in range(0,num_layers-2):
        W = weights[l]
        b = biases[l]
#         W = np.array(weights[l])
#         b = np.array(biases[l])
#         W=W.reshape(layers[l],layers[l+1])
#         b=b.reshape(layers[l],layers[l+1])
        H = tf.tanh(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
#     W=W.reshape(layers[-2],layers[-1])
    b = biases[-1]
#     b=b.reshape(layers[-1],layers[-1])
    Y = tf.add(tf.matmul(H, W), b)
    return Y

In [61]:
a=tf.Variable(x_star, dtype = tf.float32)

In [154]:
b=x_star.astype('float32')

In [155]:
H = 2.0*(b - lb)/(ub - lb) - 1.0

In [148]:
H.shape

TensorShape([Dimension(257), Dimension(1)])

In [143]:
H.dtype

dtype('float64')

In [68]:
W=[0.2,0.5]

In [74]:
W=np.array(W,dtype='float32')

In [76]:
W=W.reshape(1,2)

In [28]:
b=[0,0]

In [29]:
b=np.array(b,dtype='float32')

In [30]:
b=b.reshape(1,2)

In [77]:
c=tf.matmul(H, W)

In [78]:
c.shape

TensorShape([Dimension(257), Dimension(2)])

In [33]:
z=tf.add(c, b)

In [35]:
print(z.shape)

(257, 2)


In [94]:
W=np.array(x[0])

In [98]:
W

array(<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[-0.62955666,  1.1757891 ]], dtype=float32)>,
      dtype=object)

In [ ]:
#u net

In [113]:
def net_u(x):  
    u = neural_net(tf.concat([x],1),weights,biases)
    return u
        

In [ ]:
#net f

In [ ]:
def net_f(x):
           
    u = self.net_u(x)
    
    u_x = tf.gradients(u, x)[0]
    u_xx = tf.gradients(u_x, x)[0]
    f = -1*lambda1*u_xx
    

    return f

In [82]:
#running code
f=T_star
g=T_star

In [83]:
for i in range(100):
    f=np.append(f,g)

In [88]:
f=f.reshape(25957,1)

In [89]:
c=c.reshape(25957,1)

In [97]:
if __name__ == "__main__": 
     
#     nu = 0.01/np.pi

#     N_u = 2000
#     layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
    
#     data = scipy.io.loadmat(r'C:\Users\Omkar\Downloads\burgers_shock.mat')
    
#     t = data['t'].flatten()[:,None]
#     x = data['x'].flatten()[:,None]
#     Exact = np.real(data['usol']).T
    
#     X, T = np.meshgrid(x,t)
    
#     X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
#     u_star = Exact.flatten()[:,None]              

#     # Doman bounds
#     lb = X_star.min(0)
#     ub = X_star.max(0)    
    
#     ######################################################################
#     ######################## Noiseles Data ###############################
#     ######################################################################
#     noise = 0.0            
             
#     idx = np.random.choice(X_star.shape[0], N_u, replace=False)
#     X_u_train = X_star[idx,:]
#     u_train = u_star[idx,:]
    
    model = PhysicsInformedNN2(x_star,T_star, layers, lb, ub)
    model.train(1000)
    
#     u_pred, f_pred = model.predict(X_star)
            
#     error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
    
#     U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
        
    lambda_1_value = model.sess.run(model.lambda_1)
#     lambda_2_value = model.sess.run(model.lambda_2)
#     lambda_2_value = np.exp(lambda_2_value)
    
#     error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
#     error_lambda_2 = np.abs(lambda_2_value - nu)/nu * 100
    
#     print('Error u: %e' % (error_u))    
#     print('Error l1: %.5f%%' % (error_lambda_1))                             
#     print('Error l2: %.5f%%' % (error_lambda_2))  

Device mapping:

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles. For reference, the tensor object was Tensor("strided_slice:0", shape=(257, 1), dtype=float32) which was passed to the feed with key Tensor("Placeholder:0", shape=(?, 1), dtype=float32).

In [61]:
print(lambda_1_value)

[0.00247875]


class PhysicsInformedNN_rod:
    # Initialize the class
    def __init__(self, X, u, layers, lb, ub):
        
        self.lb = lb
        self.ub = ub
        
        self.x = X[:,0:1]
        self.u = u
        
        self.layers = layers
        
        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        # Initialize parameters
        self.lambda_1 = tf.Variable([0.0], dtype=tf.float32)
        
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        
        self.u_pred = self.net_u(self.x_tf)
        self.f_pred = self.net_f(self.x_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
    
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
        
    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
        
        
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1

        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
        
    def net_u(self, x):  
        u = self.neural_net(tf.concat([x],1), self.weights, self.biases)
        return u

    def net_f(self, x):
        lambda_1 = self.lambda_1        
        u = self.net_u(x)
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        f = -1*lambda_1*u_xx

        return f
        
        
                
        
        
    
    
    
    def callback(self, loss, lambda_1):
        print('Loss: %e, l1: %.5f' % (loss, lambda_1))
        
        
    def train(self, nIter):
        tf_dict = {self.x_tf: self.x,self.u_tf: self.u}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                lambda_1_value = self.sess.run(self.lambda_1)
                print('It: %d, Loss: %.3e, Lambda_1: %.3f, Time: %.2f' % 
                      (it, loss_value, lambda_1_value, elapsed))
                start_time = time.time()
        
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss, self.lambda_1],
                                loss_callback = self.callback)
        
        
    def predict(self, X_star):
        
        tf_dict = {self.x_tf: X_star[:,0:1]}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        f_star = self.sess.run(self.f_pred, tf_dict)
        
        return u_star, f_star

In [62]:
dataframe1 = pd.read_csv(r"C:\Users\Omkar\Downloads\Rod data.csv")
x_values=dataframe1.loc[:,"Length [mm]"] 
x_values=np.array(x_values)
x_values=x_values.reshape(257,1)
T_values=dataframe1.loc[:,"Value [K]"] 
T_values=np.array(T_values)
T_values=T_values.reshape(257,1)
x_star=x_values
# x_star=tf.Variable(x_star, dtype = tf.float32)
T_star=T_values
# T_star=tf.Variable(T_star, dtype = tf.float32)
lb = np.array([0.])
ub = np.array([0.3])

In [6]:
N_u = 200
layers = [1, 2 , 2 , 1]


In [64]:
noise = 0.0            
# Initialize parameters
lambda_1 = tf.Variable([17], dtype=tf.float32)
idx_ = np.random.choice(x_star.shape[0],N_u , replace=False)
x_u_train = x_star[idx_,:]
u_train = T_star[idx_,:]

In [4]:
def xavier_init(size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)

In [5]:
def initialize_NN(layers):
        weights = []
        biases = []
        
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases

In [7]:
weights, biases = initialize_NN(layers)

In [9]:
biases

[<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[0., 0.]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[0., 0.]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[0.]], dtype=float32)>]

In [10]:
def neural_net(X, weights, biases):
    num_layers = len(weights) + 1

    H = 2.0*(X - lb)/(ub - lb) - 1.0
    for l in range(0,num_layers-2):
        W = weights[l]
        b = biases[l]
        H = tf.tanh(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H, W), b)
    return Y

In [6]:
y=tf.Variable([1,2,3,4])
x=tf.Variable([1,2,3,4])

In [10]:
a=tf.gradients(y,x)[0]

In [11]:
import tensorflow as tf
import numpy as np

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    a_value = sess.run(y)
    b_value = sess.run(x)
    c_value= sess.run(a)
    print(c_value)
    print(b_value)

TypeError: Fetch argument None has invalid type <class 'NoneType'>

RuntimeError: Attempted to use a closed Session.

In [69]:
def net_u(x):  
        u = neural_net(tf.concat([x],1),weights,biases)
        return u

def net_f(x):        
    u = net_u(x)
    u_x = tf.gradients(u, x)[0]
    u_xx = tf.gradients(u_x, x)[0]
    f = -1*lambda_1*u_xx

    return f

In [70]:
# tf placeholders and graph
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                             log_device_placement=True))




x_tf = tf.placeholder(tf.float32, shape=[None, a.shape[1]])
u_tf = tf.placeholder(tf.float32, shape=[None, b.shape[1]])

u_pred = net_u(x_tf)
f_pred = net_f(x_tf)

loss = tf.reduce_mean(tf.square(u_tf - u_pred)) + \
            tf.reduce_mean(tf.square(f_pred))

optimizer = tf.contrib.opt.ScipyOptimizerInterface(loss, 
                                                        method = 'L-BFGS-B', 
                                                        options = {'maxiter': 50000,
                                                                   'maxfun': 50000,
                                                                   'maxcor': 50,
                                                                   'maxls': 50,
                                                                   'ftol' : 1.0 * np.finfo(float).eps})

optimizer_Adam = tf.train.AdamOptimizer()
train_op_Adam = optimizer_Adam.minimize(loss)

init = tf.global_variables_initializer()
sess.run(init)

Device mapping:



In [71]:
def callback(loss, lambda_1):
        print('Loss: %e, l1: %.5f' % (loss, lambda_1))

In [72]:
def train(nIter):
    tf_dict = {x_tf: x_star,u_tf: T_star}

    start_time = time.time()
    for it in range(nIter):
        sess.run(train_op_Adam, tf_dict)

        # Print
        if it % 10 == 0:
            elapsed = time.time() - start_time
            loss_value = sess.run(loss, tf_dict)
            lambda_1_value = sess.run(lambda_1)
            print('It: %d, Loss: %.3e, Lambda_1: %.3f, Time: %.2f' % 
                  (it, loss_value, lambda_1_value, elapsed))
            start_time = time.time()

    optimizer.minimize(sess,
                            feed_dict = tf_dict,
                            fetches = [loss, lambda_1],
                            loss_callback = callback)

In [73]:
def predict(X_star):
        
    tf_dict = {x_tf: x_star}

    u_star = sess.run(u_pred, tf_dict)
    f_star = sess.run(f_pred, tf_dict)

    return u_star, f_star

In [16]:
a=tf.Variable(x_u_train, dtype = tf.float32)
b=tf.Variable(u_train, dtype = tf.float32)
model = PhysicsInformedNN_rod(a, u_train, layers, lb, ub)

Device mapping:

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [74]:
train(1000000)

It: 0, Loss: 1.292e+06, Lambda_1: 16.999, Time: 2.00
It: 10, Loss: 1.290e+06, Lambda_1: 16.993, Time: 0.47
It: 20, Loss: 1.284e+06, Lambda_1: 16.987, Time: 0.48
It: 30, Loss: 1.272e+06, Lambda_1: 16.978, Time: 0.48
It: 40, Loss: 1.268e+06, Lambda_1: 16.974, Time: 0.48
It: 50, Loss: 1.265e+06, Lambda_1: 16.972, Time: 0.48
It: 60, Loss: 1.262e+06, Lambda_1: 16.972, Time: 0.48
It: 70, Loss: 1.260e+06, Lambda_1: 16.972, Time: 0.47
It: 80, Loss: 1.257e+06, Lambda_1: 16.971, Time: 0.59
It: 90, Loss: 1.255e+06, Lambda_1: 16.971, Time: 0.81
It: 100, Loss: 1.252e+06, Lambda_1: 16.971, Time: 0.76
It: 110, Loss: 1.250e+06, Lambda_1: 16.971, Time: 0.74
It: 120, Loss: 1.247e+06, Lambda_1: 16.971, Time: 0.75
It: 130, Loss: 1.245e+06, Lambda_1: 16.971, Time: 0.74
It: 140, Loss: 1.243e+06, Lambda_1: 16.971, Time: 0.75
It: 150, Loss: 1.240e+06, Lambda_1: 16.971, Time: 0.75
It: 160, Loss: 1.238e+06, Lambda_1: 16.971, Time: 0.75
It: 170, Loss: 1.236e+06, Lambda_1: 16.971, Time: 0.75
It: 180, Loss: 1.234e

KeyboardInterrupt: 

In [168]:
# nu = 0.01/np.pi

# N_u = 2000
# layers = [1, 20, 20, 20, 20, 20, 20, 20, 20, 1]


# data = scipy.io.loadmat(r'C:\Users\Omkar\Downloads\burgers_shock.mat')

# t = data['t'].flatten()[:,None]
# x = data['x'].flatten()[:,None]
# Exact = np.real(data['usol']).T

# X, T = np.meshgrid(x,t)

# X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
# u_star = Exact.flatten()[:,None]              

# Doman bounds


######################################################################
######################## Noiseles Data ###############################
######################################################################
# noise = 0.0            

# idx_ = np.random.choice(x_star.shape[0],N_u , replace=False)
# x_u_train = x_star[idx_,:]
# u_train = u_star[idx_,:]

model = PhysicsInformedNN(X_u_train, u_train, layers, lb, ub)
# model.train(0)

# u_pred, f_pred = model.predict(X_star)

# error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)

# U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')

# lambda_1_value = model.sess.run(model.lambda_1)
# lambda_2_value = model.sess.run(model.lambda_2)
# lambda_2_value = np.exp(lambda_2_value)

# error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
# error_lambda_2 = np.abs(lambda_2_value - nu)/nu * 100

# print('Error u: %e' % (error_u))    
# print('Error l1: %.5f%%' % (error_lambda_1))                             
# print('Error l2: %.5f%%' % (error_lambda_2))  

In [157]:
data = scipy.io.loadmat(r'C:\Users\Omkar\Downloads\burgers_shock.mat')
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]  

In [158]:
X_star.shape

(25600, 2)